In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np 
import pandas as pd 

In [ ]:
!pip install py7zr

In [ ]:
from py7zr import unpack_7zarchive
import shutil

shutil.register_unpack_format('7zip', ['.7z'], unpack_7zarchive)

In [ ]:
shutil.unpack_archive('/kaggle/input/cifar-10/train.7z', '/kaggle/temp/')

In [ ]:
train_labels=pd.read_csv("/kaggle/input/cifar-10/trainLabels.csv", header='infer')

classes=train_labels['label'].unique()

import os

if not os.path.exists("/kaggle/temp/validation/"):
    os.mkdir("/kaggle/temp/validation/")

parent_path_train="/kaggle/temp/train/"
parent_path_validation="/kaggle/temp/validation/"
parent_path_test="/kaggle/temp/test/"

for class1 in classes:
    path_train=os.path.join(parent_path_train,class1)
    if not os.path.exists(path_train):
        os.mkdir(path_train)
    path_validation=os.path.join(parent_path_validation,class1)
    if not os.path.exists(path_validation):
        os.mkdir(path_validation)

for (int_ind,row) in train_labels.iterrows():
    id=str(row["id"])+".png" 
    source_path=os.path.join(parent_path_train,id)
    
    p=np.random.random() 
    if p <= 0.8: 
        target_path=os.path.join(parent_path_train,row["label"],id)
        os.replace(source_path, target_path) 
    else:
        target_path=os.path.join(parent_path_validation,row["label"],id)
        os.replace(source_path, target_path)

In [ ]:
os.listdir('/kaggle/temp/train'),os.listdir('/kaggle/temp/validation'),len(os.listdir('/kaggle/temp/train/horse'))

In [ ]:
import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.applications.xception import Xception, preprocess_input
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Activation, BatchNormalization, GlobalAveragePooling2D,Conv2D,Flatten
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ReduceLROnPlateau

from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
def my_model3():
    model=Sequential()
    model.add( Conv2D(filters=32, kernel_size=(3,3), strides=(1,1), padding='valid', activation=None, use_bias=False, input_shape=(32,32,3)) )
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    
    model.add( Conv2D(filters=48, kernel_size=(3,3), strides=(1,1), padding='valid', activation=None, use_bias=False) )
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    
    model.add( Conv2D(filters=64, kernel_size=(3,3), strides=(1,1), padding='valid', activation=None, use_bias=False) )
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    
    model.add( Conv2D(filters=80, kernel_size=(3,3), strides=(1,1), padding='valid', activation=None, use_bias=False) )
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    
    model.add( Conv2D(filters=96, kernel_size=(3,3), strides=(1,1), padding='valid', activation=None, use_bias=False) )
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    
    model.add( Conv2D(filters=112, kernel_size=(3,3), strides=(1,1), padding='valid', activation=None, use_bias=False) )
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    
    model.add( Conv2D(filters=128, kernel_size=(3,3), strides=(1,1), padding='valid', activation=None, use_bias=False) )
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    
    model.add( Conv2D(filters=144, kernel_size=(3,3), strides=(1,1), padding='valid', activation=None, use_bias=False) )
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    
    model.add( Conv2D(filters=160, kernel_size=(3,3), strides=(1,1), padding='valid', activation=None, use_bias=False) )
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    
    model.add( Conv2D(filters=176, kernel_size=(3,3), strides=(1,1), padding='valid', activation=None, use_bias=False) )
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    
    model.add(Flatten())
    
    model.add(Dense(units=10))
    model.add(BatchNormalization())
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model


In [ ]:
model3=my_model3()

In [ ]:
reduceLROnPlateau = ReduceLROnPlateau(monitor='val_acc', 
                                patience=3,
                                verbose=1, 
                                factor=0.5,
                                min_lr=0.00001)

In [ ]:
train_datagen=ImageDataGenerator(featurewise_center=False,
                         samplewise_center=False,
                         featurewise_std_normalization=False,
                         samplewise_std_normalization=False,
                         zca_whitening=False,
                         rotation_range=10,
                         zoom_range=0.1,
                         width_shift_range=0.1,
                         height_shift_range=0.1,
                         horizontal_flip=False,
                         vertical_flip=False,
                         rescale=1./255
                        )
train_gen=train_datagen.flow_from_directory(directory='/kaggle/temp/train/', target_size=(32,32), batch_size=32) 

validation_datagen=ImageDataGenerator(rescale=1./255)

valid_gen=validation_datagen.flow_from_directory(directory='/kaggle/temp/validation/', target_size=(32,32), batch_size=32)

In [ ]:
model3.fit(train_gen, epochs=75, callbacks=[reduceLROnPlateau], validation_data=valid_gen, steps_per_epoch = train_gen.n//train_gen.batch_size,
                    validation_steps = valid_gen.n//valid_gen.batch_size)    

In [ ]:
shutil.unpack_archive('/kaggle/input/cifar-10/test.7z', '/kaggle/temp/test')

In [ ]:
# shutil.unregister_unpack_format('7zip', ['.7z'], unpack_7zarchive)

In [ ]:
test_datagen=ImageDataGenerator(rescale=1./255)

test_gen=test_datagen.flow_from_directory(directory='/kaggle/temp/test', target_size=(32,32), batch_size=64, class_mode=None, shuffle=False)

In [ ]:
model3.evaluate(test_gen)

In [ ]:
test_gen.reset()
prediction_vectors3=model3.predict(test_gen)
predictions_final=np.argmax(prediction_vectors3, axis=1)

In [ ]:
sub=pd.read_csv('/kaggle/input/cifar-10/sampleSubmission.csv', header='infer')

sub['label']=predictions_final


sub.to_csv('/kaggle/working/submission.csv', index=False)

In [ ]:
# pred = np.argmax(model.predict(testflow), axis = 1)
pred = list(predictions_final)
classes = {value:key for (key, value) in train_gen.class_indices.items()}
predc = np.empty(300000, 'U16')
idd = 0
for i in pred:
    predc[idd] = classes[i]
    idd += 1



files = []
for file in test_gen.filenames:
    files.append(int(file.split('/')[1].split('.')[0]) - 1)
files = np.array(files)
    
final = np.empty(300000, dtype = 'U16')
final[files] = predc




sub = pd.read_csv('../input/cifar-10/sampleSubmission.csv')
sub['label'] = final
sub.to_csv('submission.csv', index = False)